# NumPy Linear Algebra Lab  
### A Practical Math Engine for Machine Learning Systems

This repository is not a collection of NumPy examples.  
It is a **designed math layer** that implements, tests, and validates the core linear algebra operations used inside real machine learning and optimization pipelines.

The goal is simple:  
> Treat linear algebra as **software infrastructure**, not as theory.

---

## System Overview

The project is organized as a modular math engine where each layer represents a real engineering responsibility:



In [1]:
import numpy as np
from numpy import linalg  as LA

In [2]:
A  = np.array([[1 ,2 ,3] , [np.nan , 4,5], [0 , 8 , 9]])
A

array([[ 1.,  2.,  3.],
       [nan,  4.,  5.],
       [ 0.,  8.,  9.]])

In [3]:
A.shape

(3, 3)

In [4]:
A.ndim #Dimensions of the array

2

In [6]:
A.size #total numbers inside the array

9

In [7]:
print(np.isfinite(A))

[[ True  True  True]
 [False  True  True]
 [ True  True  True]]


## Basic Building Blocks

## `np.diag` - Create or extract diagonal

In [8]:
np.diag(A)

array([1., 4., 9.])

# `np.triu()` and  `np.tril()` - Extract Lower and Upper triangle of the matrix , Used in - Optimization, numerical solvers

In [9]:
np.triu(A)

array([[1., 2., 3.],
       [0., 4., 5.],
       [0., 0., 9.]])

In [10]:
np.tril(A)

array([[ 1.,  0.,  0.],
       [nan,  4.,  0.],
       [ 0.,  8.,  9.]])

## `np.block` - Used to Combine Matrices , Used in Big system Modelling

In [12]:
np.block([[A,A],[A,A]])

array([[ 1.,  2.,  3.,  1.,  2.,  3.],
       [nan,  4.,  5., nan,  4.,  5.],
       [ 0.,  8.,  9.,  0.,  8.,  9.],
       [ 1.,  2.,  3.,  1.,  2.,  3.],
       [nan,  4.,  5., nan,  4.,  5.],
       [ 0.,  8.,  9.,  0.,  8.,  9.]])

## Measurement tools

`LA.norm()` - measures how big is the size of the matrix . In ML it measures Error Size or weight Strength

In [17]:
print(LA.norm(A)) #sqrt(1² + 2² + 3² + nan² + 4² + 5² + 0² + 8² + 9²) hence nan

nan


`LA.cond()`- Tells how numerically stable is our matrix , it tells - “If my input data changes a tiny bit, will my output change a tiny bit or a LOT?”
| Value     | Meaning                          |
| --------- | -------------------------------- |
| `~1`      | Very stable, well-conditioned    |
| `10–1000` | Acceptable                       |
| `> 10⁶`   | Dangerous (numerically unstable) |
| `inf`     | Singular matrix (non-invertible) |


In [20]:
A  = np.array([[1 ,2 ,3] , [6, 4,5], [0 , 8 , 9]])
print(LA.cond(A))

33.838184173990314


Interpretation

A condition number of ~34 means:

If your input data has a 1% error, your output could have up to 34% error in the worst case.

## `LA.cond(A, 1) — Column Sensitivity`

In [ ]:
#How sensitive is my system to changes in the columns of A?
#This uses the 1-norm, which is: Maximum column sum
LA.cond(A, 1)

np.float64(56.312500000000014)

## `LA.cond(A, ∞) — Row Sensitivity`

In [23]:
#This uses the infinity norm, which is: Maximum row sum
#“How sensitive is my system to changes in the columns of A?”
LA.cond(A, np.inf)  

np.float64(40.37500000000001)

# `np.linalg.slogdet()` - Safe determinant  for very large numbers


## Function Purpose
`np.linalg.slogdet(A)` computes the determinant of a matrix in a **numerically stable way**.

Instead of returning `det(A)` directly (which can overflow or underflow for large matrices), it returns:

`(sign, log(|det(A)|))`

Meaning of Each Value
1. sign = 1.0

This tells you the sign of the determinant:

Value	Meaning
+1	det(A) > 0 → Orientation preserved
-1	det(A) < 0 → Orientation flipped
0	det(A) = 0 → Singular (not invertible)

2. logabsdet = 3.4657

This is:

    ln(|det(A)|)


To get the actual determinant:

    detA = np.exp(3.465735902799726)


Approximate result:

    det(A) ≈ 32

`Geometric Interpretation`

    The determinant represents how much the matrix scales volume.

    This matrix transforms a unit cube in 3D space into a shape with ~32× the volume.


`Practical Interpretation`

Since:

    sign = +1 → No spatial flip

    det(A) ≈ 32 → Strong scaling, not collapsing

This means:

    The matrix is numerically healthy and safe to invert

    This aligns with:

LA.cond(A) ≈ 33.8


Which indicates moderate but acceptable sensitivity.

In [25]:
print(np.linalg.slogdet(A))

SlogdetResult(sign=np.float64(1.0), logabsdet=np.float64(3.465735902799726))


In [26]:
print(np.linalg.det(A))
print(np.exp(np.linalg.slogdet(A)[1]))


31.999999999999986
31.999999999999986


## Solving Systems

## `LA.solve()` - Finding the unknowns 
- Solving the equations such as AX = B

In [29]:
b = np.array([[10 , 20 , 30] , [40 , 50 , 60] , [70 , 80 , 90]])
x = LA.solve(A, b)
x


array([[  1.875 ,   1.875 ,   1.875 ],
       [ 22.8125,  12.8125,   2.8125],
       [-12.5   ,  -2.5   ,   7.5   ]])

## `LA.lstsq()` - It solves this problem:

“Find x that makes Ax ≈ b as close as possible,
even when there is no exact solution.”

Instead of solving:

    Ax = b


It solves:

    minimize ||Ax - b||²


This is called Least Squares.

`When use it`:

    Use lstsq() when:

        - A is not square (more equations than unknowns)

        - A is noisy

        - Data is inconsistent

        - Regression / fitting / ML

## Decompositions

“Break matrices into powerful parts”

`Eigenvalues & Eigenvectors`

In [34]:
vals, vecs = LA.eig(A)

# `vals` — Eigenvalues (λ)

These are scaling factors.

Each value tells you:

Along this special direction, A stretches or shrinks space by this much.

In [33]:
vals

array([-0.18197634+1.48144647j, -0.18197634-1.48144647j,
       14.36395268+0.j        ])

# `vecs` — Eigenvectors (v)

These are the special directions.

Each column in vecs is an eigenvector that matches the eigenvalue at the same index in vals.

So:

vals[i]  ↔  vecs[:, i]

In [32]:
vecs

array([[ 0.0077823 +0.2640235j ,  0.0077823 -0.2640235j ,
         0.26048212+0.j        ],
       [ 0.73120443+0.j        ,  0.73120443-0.j        ,
         0.53767419+0.j        ],
       [-0.62091468-0.10018016j, -0.62091468+0.10018016j,
         0.80190743+0.j        ]])

# `LA.eigh()`

-Faster, safer eigen solver for symmetric matrices

In [35]:
LA.eigh(A)

EighResult(eigenvalues=array([-5.71282033,  3.89545215, 15.81736819]), eigenvectors=array([[ 0.61758983, -0.74434797,  0.2540254 ],
       [-0.69096159, -0.35920399,  0.62733131],
       [ 0.37570585,  0.56295523,  0.73615659]]))

`Singular Value Decomposition (SVD)`
-Used in:

PCA

Image compression

Recommendation systems

In [36]:
U , S , Vt = LA.svd(A)

`QR Decomposition`

In [ ]:
Q , R  = LA.qr(A)